# Stock Data and Sentiment Analysis Agent Workflow (with Phidata)

## What is Phidata?

Phidata is a framework for building agentic systems, developers can use phidata to:
* Build Agents with memory, knowledge and external connections
* Build a team of agents that can work together
* Monitor, evaluate and optimize Agents

Phidata also offers an intutive and user-friendly Agent UI that enables users to create Agents with a visual interface. Check it out here: https://www.phidata.com/


## Core Development Goals for this Project:


**AI Agentic Flow:**
* Create AI Agents that are autonomously driven for specific tasks that can follow rules and adapt to input
* Agentic orchestration should require minimal human input
* Test and iterate different tool combination including Google search + yfinance



## Install and Instantiate Dependencies

In [1]:
!pip install phidata openai anthropic yfinance googlesearch-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.4/714.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 7.0 MB/s eta 0:00:00


In [2]:
## Require library for working with real-time Google search news

!pip install pycountry -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.4 MB/s eta 0:00:00


The workflow is implemented in Python using several libraries:

* Phidata: An open-source library for building and managing data pipelines.
* OpenAI and/or Claude: Provides access to the base LLM for use.
* yfinance: Used to fetch financial data from Yahoo Finance.
* Google Search: Used to search the web for news articles.

In [3]:
## Libraries

from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.model.anthropic import Claude
from phi.tools.yfinance import YFinanceTools
from phi.tools.googlesearch import GoogleSearch

## Define the LLM to Use

* Option to switch between OpenAI or Anthropic
* In my experiments, I have found that Claude 3.5. Sonnett works better with structured data inputs from financial APIs.
* OpenAI GPT Models provide better unstructured outputs

**Note:** Users should try to use different models for different agent tasks. Try instantiating Claude for data analysis tasks and GPT for sentiment analysis + writing tasks

In [4]:
import requests

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY


## Define Agents and Tools

### Phidata Agent Tools

The `phi.tools` module in Phidata provides a collection of pre-built tools that agents can use to interact with external services and data sources. These tools simplify the process of making tool calls, allowing agents to focus on their core tasks.

Here we use a combination of GoogleSearch and YFinance calls to create a 360 degree view of a company.

#### Sentiment Analysis Agent

In [5]:
# Sentiment Analysis Agent

sentiment_agent = Agent(
    name="Sentiment Analysis Agent",
    role="Search and interpret news articles",
    model=OpenAIChat(id="gpt-4o"),
    tools=[GoogleSearch(), YFinanceTools(company_news=True)],
    instructions=[
        "Find relevant news articles for each company and critically analyze the news sentiment.",
        "Provide sentiment scores from 1 (negative) to 10 (positive) with reasoning and sources."
        "Cite your sources. Be specific, crtical and provide relevant links."
    ],
    show_tool_calls=True,
    markdown=True,
)

#### Basic Stock Info Agent

In [6]:
# Financial Analyst Agent

basic_stock_agent = Agent(
    name="Basic Financial Data Agent",
    role="Retrieve basic company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=Claude(id="claude-3-5-sonnet-latest"),
    tools=[YFinanceTools(stock_price=True, company_info=True, stock_fundamentals=True, analyst_recommendations=True)],
    instructions=[
        "Retrieve stock prices, analyst recommendations, and key summary financial data.",
        "Focus on company funamentals and trends, presenting the data in tables with key insights."
    ],
    show_tool_calls=True,
    markdown=True,
)

#### Historical Financials Agent

In [7]:
# Historical Data Analyst Agent

historical_financials_agent = Agent(
    name="Comprehensive Financial Data Agent",
    role="Retrieve comprehensive company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=Claude(id="claude-3-5-sonnet-latest"),
    tools=[YFinanceTools(company_info=True, key_financial_ratios=True, income_statements=True, historical_prices=True)],
    instructions=[
        "Retrieve key financial ratios, income statement, balance sheet, cash flow statements and historical price data.",
        "Focus on company fundamentals and trends, presenting the data in tables with key insights."
    ],
    show_tool_calls=True,
    markdown=True,
)

#### Executive Agent


*   Serves as the orchestration agent - defined as a portfolio manager.
*   Consolidates all the information collected by other tool-wielding agents and provides a BUY, SELL or HOLD recommendation



In [8]:
# Data Synthesis / Executive Agent

executive_agent = Agent(
    name="Executive Agent",
    role="You are a seasoned portfolio manager and executive. You are tasked with making data-driven decisions and drawing conclusions through critical assessment",
    model=Claude(id="claude-3-5-sonnet-latest"),
    instructions=[
        "Check outputs for accuracy and completeness.",
        "Synthesize data to provide a final sentiment score (1-10) with justifcation."
        "Finally, after critical reasoning, provide a BUY, SELL or HOLD recommendation for the stock."
    ],
    show_tool_calls=True,
    markdown=True,
)

#### Quantitative Finance Agent (with Python) - **Use optionally and with caution**

**Note:** The quant agent can run into issues with recurring scripts and errors saving / reading files.


*   This Agent will save scripts into your local instance.
*   More proficient when run in local IDE or WSL environment.
*   GPT 4o handles code execution less proficiently than Claude 3.5 Sonnet

In [9]:
# Quant Finance Agent (Optional)

from phi.tools.python import PythonTools

quant_agent = Agent(
    name="Quant Finance Agent",
    role="You are a seasoned quantitative financial analyst. You are tasked with conducting data-driven python analysis on stock information",
    model=Claude(id="claude-3-5-sonnet-latest"),
    tools=[PythonTools()],
    instructions=[
        "Use advanced quantitative and other machine learning learning techniques to critically analyze the company's financials.",
        "You will use Python and other sophisticated quantitiative finance, Data science, and machine learning techniques to critically analyze companies based on the queries."
        "All techniques used should provide a comprehensive and critical view of the company, this includes valuation, evaluating news sentiment, and understanding core company fundamentals"
        "You should expertly breakdown each methodology used and ensure you are arriving at conclusive results."
    ],
    show_tool_calls=True,
    markdown=True,
)


## Create the Multi Agent Orchestration (Build Team)

These few lines of code set up a sophisticated multi agent system with specific roles that collaborate to analyze a stock.

The `agent_team` acts as a conductor, picking the right agent work flow and ensuring that each agent contributes its insights and that the final output is well-structured, data-driven, and easily understandable

In [10]:
# Full Team of Agents
from datetime import date

today = date.today().strftime("%Y-%m-%d")

agent_team = Agent(
    model=Claude(id="claude-3-5-sonnet-latest"),
    team=[sentiment_agent, basic_stock_agent, historical_financials_agent, executive_agent], ## add quant_agent here if desired.
    instructions=[
        "Combine the expertise of all the agents part of your team to provide a cohesive, highly analytical and well-supported response",
        f"Always include references and dates for all data points and sources. Your context is current as of {today}"
        "Present data in structured tables for clarity, ensuring all conclusions are data-driven."
        "Explain the methodlogy and always determine a BUY, SELL, or HOLD recommendation."
    ],
    show_tool_calls=True,
    markdown=True,
)

In [11]:
agent_team.print_response(
    "Analyze the sentiment for the following companies for the month of December 2024 (your context is up-to-date as of Jan 16 2025): JPM, META. \n\n"
    "1. **Sentiment Analysis**: Search for relevant news articles and interpret the sentiment for each company. Provide the sentiment score"
    "2. **Financial Data**: Analyze stock price and company fundamental financial metrics and other resources. Highlight key trends"
    "3. **Consolidated Analysis**: Combine the insights from the various srouces such as sentiment analysis and financial data to assign a final BUY, SELL or HOLD recommendation"
    "Ensure your response is accurate, comprehensive, analytical and well-reasoned. Include references to sources with publication dates.",
    stream=True
)

Output()

In [12]:
agent_team.print_response(
    "Perform a discounted cash flow analysis of META stock to assess the stock's intrinsic value relative to its current price \n\n"
    "1. **Financial Data**: Analyze stock price and company fundamental financial metrics and other resources. Highlight key trends"
    "2. **Consolidated Analysis**: Combine the insights from the various srouces such as sentiment analysis and financial data to assign a final BUY, SELL or HOLD recommendation"
    "Ensure your response is accurate, comprehensive, analytical and well-reasoned. Include references to sources with publication dates.",
    stream=True
)

Output()

#### Testing Different Model Outputs

In [13]:
# Full Team of Agents

agent_team = Agent(
    model=Claude(id="claude-3-5-haiku-latest"), ## using Claude Haiku instead of Sonnett (faster, lower cost option - quality is comparable)
    team=[sentiment_agent, basic_stock_agent, historical_financials_agent, executive_agent], ## add quant_agent here if desired.
    instructions=[
        "Combine the expertise of all the agents part of your team to provide a cohesive, highly analytical and well-supported response",
        "Always include references and dates for all data points and sources."
        "Present data in structured tables for clarity, ensuring all conclusions are data-driven."
        "Explain the methodlogy and always determine a BUY, SELL, or HOLD recommendation."
    ],
    show_tool_calls=True,
    markdown=True,
)

In [14]:
agent_team.print_response(
    "Perform a discounted cash flow analysis of Cal maine foods stock (ticker: CALM) to assess the stock's intrinsic value relative to its current price \n\n"
    "1. **Financial Data**: Analyze stock price and company fundamental financial metrics and other resources. Highlight key trends"
    "2. **Consolidated Analysis**: Combine the insights from the various srouces such as sentiment analysis and financial data to assign a final BUY, SELL or HOLD recommendation"
    "Ensure your response is accurate, comprehensive, analytical and well-reasoned. Include references to sources with publication dates.",
    stream=True
)

Output()

In [15]:
agent_team.print_response(
    "Perform a public comparable companies analysis for CALM Cal Maine Foods. Identify it's main competitors and market positioning. Assess each comp in a structrued table format",
    stream=True
)

Output()

## Key Takeaways

### Results


*   Using a combination of different models improve output quality.
*   We find Claude 3.5 Sonnet excels in parsing through structured data
*   GPT 4o works well for understanding news sentiment and proucing copy.



### Limitations

* Output quality and control requires effective prompt engineering
* Base models default to their cut-off dates, requires a current date input or assumption in base prompt
* Computation time - depends on system capabilities, query complexity, number of Agents being used, etc.